# Final CHO Model
This notebook is to asses the validity of our reconstruction and how complete it is.

[1. Generation of the dataset and model reconstruction](#generation) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.1 Retrieve information from the Google Sheet datasets reactions and metabolites**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.2 Build a model and feed it the information from the df generated** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.3 Save and validate the model** <br>

[2. Identification of Blocked Reactions and Dead-End Metabolites](#blocked&deadends) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.1 Retrieve information from the Google Sheet datasets reactions and metabolites**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.2 Build a model and feed it the information from the df generated** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.3 Save and validate the model** <br>

[3. Generation of the Mass Flow Graph](#MFG) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**3.1 Generation of the "D-Matrix"**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**3.2 Plotting the "D-Matrix", Normalized Flow Graph (NFG)** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**3.3 Generation of the "FluxOpenValue" matrix** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**3.4 Generation of the "Mass Flow Graph (MFG) Matrix"** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**3.5 Plotting the "MFG Matrix"** <br>

## 1. Generation of the dataset and model reconstruction <a id='generation'></a>
Here we generate the CHO model from the dataset stored in the Google Sheet file. We first use the google_sheet module to extract all the necessary information from the original dataset. Then we use those dataset and the COBRApy library to: (1) Create a new model and add reactions from the **Rxns Sheet**, (2) Add information on each reaction obtained from the **Rxns Sheet** and **Attributes Sheet**, (3) Add boundary reactions from the **BoundaryRxns Sheet**, and (4) Add information for each metabolite from the **Metabolites Sheet**. Finally we save the model as a SBML file and validate it using the cobrapy built-in function "validate_sbml_model( )".

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.io import validate_sbml_model, write_sbml_model
from tqdm.notebook import tqdm
from google_sheet import GoogleSheet

### 1.1 Retrieve information from the Google Sheet datasets reactions and metabolites

In [2]:
##### ----- Generate datasets from Google Sheet ----- #####

#Credential file
KEY_FILE_PATH = 'credentials.json'

# #CHO Network Reconstruction + Recon3D_v2 Google Sheet ID
# SPREADSHEET_ID = '1MlBXeHIKw8k8fZyXm-sN__AHTRSunJxar_-bqvukZws'

#CHO Network Reconstruction + Recon3D_v3 Google Sheet ID
SPREADSHEET_ID = '1MlBXeHIKw8k8fZyXm-sN__AHTRSunJxar_-bqvukZws'

# Initialize the GoogleSheet object
sheet = GoogleSheet(SPREADSHEET_ID, KEY_FILE_PATH)

# Read data from the Google Sheet
sheet_met = 'Metabolites'
sheet_rxns = 'Rxns'
sheet_attributes = 'Attributes'
sheet_boundary = 'BoundaryRxns'

metabolites = sheet.read_google_sheet(sheet_met)
rxns = sheet.read_google_sheet(sheet_rxns)
rxns_attributes = sheet.read_google_sheet(sheet_attributes)
boundary_rxns = sheet.read_google_sheet(sheet_boundary)

### 1.2 Build a model and feed it the information from the df generated

In [3]:
##### ----- Create a model and add reactions ----- #####
model = Model("iCHO")
lr = []
for _, row in rxns.iterrows():
    r = Reaction(row['Reaction'])
    lr.append(r)    
model.add_reactions(lr)
model

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24


Name,iCHO
Memory address,1521324d0
Number of metabolites,0
Number of reactions,10692
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [4]:
##### ----- Add information to each one of the reactions ----- #####
for i,r in enumerate(tqdm(model.reactions)):
    print(r.id)
    r.build_reaction_from_string(rxns['Reaction Formula'][i])
    r.name = rxns['Reaction Name'][i]
    r.subsystem = rxns['Subsystem'][i]
    r.gene_reaction_rule = str(rxns['GPR_final'][i])
    r.lower_bound = float(rxns_attributes['Lower bound'][i])
    r.upper_bound = float(rxns_attributes['Upper bound'][i])    

  0%|          | 0/10692 [00:00<?, ?it/s]

10FTHF5GLUtl
unknown metabolite '10fthf5glu_c' created
unknown metabolite '10fthf5glu_l' created
10FTHF5GLUtm
unknown metabolite '10fthf5glu_m' created
10FTHF6GLUtl
unknown metabolite '10fthf6glu_c' created
unknown metabolite '10fthf6glu_l' created
10FTHF6GLUtm
unknown metabolite '10fthf6glu_m' created
10FTHF7GLUtl
unknown metabolite '10fthf7glu_c' created
unknown metabolite '10fthf7glu_l' created
10FTHF7GLUtm
unknown metabolite '10fthf7glu_m' created
10FTHFtl
unknown metabolite '10fthf_c' created
unknown metabolite '10fthf_l' created
10FTHFtm
unknown metabolite '10fthf_m' created
11DOCRTSLte
unknown metabolite '11docrtsl_c' created
unknown metabolite 'atp_c' created
unknown metabolite 'h2o_c' created
unknown metabolite '11docrtsl_e' created
unknown metabolite 'adp_c' created
unknown metabolite 'h_c' created
unknown metabolite 'pi_c' created
11DOCRTSLtm
unknown metabolite '11docrtsl_m' created
11DOCRTSLtr
unknown metabolite '11docrtsl_r' created
11DOCRTSTRNte
unknown metabolite '11docr

5FTHFt2
unknown metabolite '5fthf_e' created
unknown metabolite 'oh1_c' created
unknown metabolite '5fthf_c' created
unknown metabolite 'oh1_e' created
5G2OXPTtx
unknown metabolite '5g2oxpt_c' created
unknown metabolite '5g2oxpt_x' created
5HLTDL
unknown metabolite '5htrp_c' created
unknown metabolite 'srtn_c' created
5HOMEPRAZOLEte
unknown metabolite '5homeprazole_e' created
unknown metabolite '5homeprazole_c' created
5HOXINDACTO2OX
unknown metabolite '5hoxindact_c' created
unknown metabolite '5hoxindoa_c' created
5HOXINDACTOX
5HOXINDACTOXm
unknown metabolite '5hoxindact_m' created
unknown metabolite '5hoxindoa_m' created
5HOXINDOAtr
unknown metabolite '5hoxindoa_e' created
unknown metabolite 'so4_c' created
unknown metabolite 'so4_e' created
5HOXINOXDA
5HTRPDOX
unknown metabolite '5hoxnfkyn_c' created
5HTRPVESSEC
unknown metabolite '5htrp_e' created
5HXKYNDCL
unknown metabolite '5hxkyn_c' created
unknown metabolite '5hxkynam_c' created
5HXKYNOXDA
unknown metabolite '46dhoxquin_c' cre

unknown metabolite 'trnaala_c' created
ALATRAHm
unknown metabolite 'alatrna_m' created
unknown metabolite 'trnaala_m' created
ALATRS
ALATRSm
ALA_DTDe
ALAt2r
ALAt2rL
unknown metabolite 'ala_L_l' created
ALAt4
ALAtN1
ALAyLATthc
unknown metabolite 'arg_L_c' created
unknown metabolite 'arg_L_e' created
ALCD1
unknown metabolite 'meoh_c' created
unknown metabolite 'fald_c' created
ALCD21_D
unknown metabolite '12ppd_R_c' created
unknown metabolite 'lald_D_c' created
ALCD21_L
unknown metabolite '12ppd_S_c' created
unknown metabolite 'lald_L_c' created
ALCD22_D
ALCD22_L
ALCD2if
unknown metabolite 'etoh_c' created
ALCD2yf
ALDD19x
unknown metabolite 'pacald_c' created
unknown metabolite 'pac_c' created
ALDD19x_P
ALDD20x
unknown metabolite 'id3acald_c' created
unknown metabolite 'ind3ac_c' created
ALDD20xm
unknown metabolite 'id3acald_m' created
unknown metabolite 'ind3ac_m' created
ALDD21
unknown metabolite 'pristanal_c' created
unknown metabolite 'prist_c' created
ALDD2x
ALDD2xm
ALDD2y
ALDSTRNte

BiGGRxn57
unknown metabolite '23doguln_c' created
BiGGRxn67
unknown metabolite 'gal1p_c' created
unknown metabolite 'ump_c' created
BiGGRxn71
BiGGRxn72
BiGGRxn76
unknown metabolite 'ha_l' created
BiGGRxn81
BiGGRxn91
unknown metabolite 'Rh3cit_c' created
unknown metabolite 'HC00342_c' created
C02528LGBAB1_r
unknown metabolite 'coa_r' created
unknown metabolite 'ppi_r' created
unknown metabolite 'dcholcoa_r' created
C02528LGBAB2_c
unknown metabolite 'dcholcoa_c' created
C02712te
unknown metabolite 'C02712_c' created
unknown metabolite 'C02712_e' created
C03681te
unknown metabolite 'C03681_c' created
unknown metabolite 'C03681_e' created
C03990ATP
unknown metabolite 'HC02191_e' created
C03990t
C04483t1
unknown metabolite 'dchac_e' created
C05299te
unknown metabolite 'C05299_c' created
unknown metabolite 'C05299_e' created
C05301te
unknown metabolite 'C05301_c' created
unknown metabolite 'C05301_e' created
C05302te
unknown metabolite 'C05302_c' created
unknown metabolite 'C05302_e' created

...

RIBFLVte
RIBt
unknown metabolite 'rib_D_e' created
RIBt2
RN0001C
unknown metabolite 'C05708_c' created
RN0001R
unknown metabolite 'selmeth_r' created
unknown metabolite 'C05708_r' created
RN0002N
unknown metabolite '5mthf_n' created
unknown metabolite 'selhcys_n' created
unknown metabolite 'selmeth_n' created
unknown metabolite 'thf_n' created
RN0002R
unknown metabolite '5mthf_r' created
unknown metabolite 'selhcys_r' created
unknown metabolite 'thf_r' created
RN0013C
RN0014R
unknown metabolite 'nrpphr_r' created
unknown metabolite 'CE5538_r' created
RN0020C
RN0020R
RN0021C
RN0021R
RN0021X
RN0022C
RN0022R
RN0022X
unknown metabolite 'C14851_x' created
unknown metabolite 'CN0010_x' created
RN0023C
RN0023R
unknown metabolite 'C14849_r' created
unknown metabolite 'CN0011_r' created
RN0023X
unknown metabolite 'C14849_x' created
unknown metabolite 'CN0011_x' created
RN0027C
RN0027R
RN0028C
RN0028R
RN0028X
unknown metabolite 'CN0021_x' created
unknown metabolite 'CN0022_x' created
RN0029C
RN0

SameRxns084
SameRxns085
SameRxns086
SameRxns087
SameRxns088
SameRxns089
unknown metabolite 'prpp_e' created
SameRxns090
SameRxns091
SameRxns092
SameRxns093
SameRxns094
SameRxns095
SameRxns096
SameRxns097
SameRxns098
SameRxns099
SameRxns100
SameRxns101
SameRxns102
SameRxns103
SameRxns104
SameRxns105
SameRxns106
SameRxns107
SameRxns108
SameRxns109
SameRxns110
SameRxns111
SameRxns112
SameRxns113
SameRxns114
SameRxns115
SameRxns116
SameRxns117
SameRxns118
SameRxns119
SameRxns120
SameRxns121
SameRxns122
SameRxns123
SameRxns124
SameRxns125
unknown metabolite 'tre_e' created
SameRxns126
SameRxns127
SameRxns128
unknown metabolite 'whtststerone_e' created
SameRxns129
SameRxns130
SameRxns131
SameRxns132
SameRxns133
unknown metabolite 'cl_x' created
SameRxns135
SameRxns136
SameRxns137
SameRxns138
SameRxns139
SameRxns140
unknown metabolite 'yvite_e' created
SameRxns141
SameRxns142
SameRxns143
SameRxns145
SameRxns146
SameRxns147
unknown metabolite 'hs_r' created
SameRxns148
SameRxns149
SameRxns150


TYRALAPHEt
unknown metabolite 'tyralaphe_e' created
unknown metabolite 'tyralaphe_c' created
TYRALAt
unknown metabolite 'tyrala_e' created
unknown metabolite 'tyrala_c' created
TYRARGGLUt
unknown metabolite 'tyrargglu_e' created
unknown metabolite 'tyrargglu_c' created
TYRARGSERt
unknown metabolite 'tyrargser_e' created
unknown metabolite 'tyrargser_c' created
TYRASE
unknown metabolite 'melanin_c' created
TYRASE2
unknown metabolite '56iqcrbxlt_c' created
TYRASPARGt
unknown metabolite 'tyrasparg_e' created
unknown metabolite 'tyrasparg_c' created
TYRATB0tc
unknown metabolite 'tyr_L_e' created
TYRB0AT3tc
TYRCBOX
TYRCYSGLYt
unknown metabolite 'tyrcysgly_e' created
unknown metabolite 'tyrcysgly_c' created
TYRCYSTHRt
unknown metabolite 'tyrcysthr_e' created
unknown metabolite 'tyrcysthr_c' created
TYRDHINDOX
TYRDOPO
TYRDOPO3
TYRDOPOX
TYRGLUt
unknown metabolite 'tyrglu_e' created
unknown metabolite 'tyrglu_c' created
TYRLEUARGt
unknown metabolite 'tyrleuarg_e' created
unknown metabolite 'tyr

unknown metabolite 'HC00576_c' created
r0466
r0470
r0472
r0474
r0475
r0480
unknown metabolite 'HC00718_c' created
r0483
r0488
r0494
unknown metabolite 'dtdp_e' created
unknown metabolite 'dtmp_e' created
r0497
unknown metabolite 'dttp_e' created
r0509
r0510
r0511
r0512
r0514
unknown metabolite 'fol_m' created
r0517
r0522
unknown metabolite '5fthf_m' created
r0523
r0525
r0527
r0531
r0537
unknown metabolite 'hxdcal_c' created
r0541
r0546
unknown metabolite 'pacald_m' created
unknown metabolite 'pac_m' created
r0548
r0553
r0555
r0556
r0557
r0558
unknown metabolite 'HC01522_c' created
unknown metabolite 'HC00460_c' created
r0559
r0560
r0568
r0570
r0571
unknown metabolite 'HC00900_m' created
r0573
unknown metabolite 'cholcoa_r' created
r0575
r0578
r0579
unknown metabolite 'ptth_m' created
unknown metabolite 'pan4p_m' created
r0580
unknown metabolite 'HC01231_c' created
r0587
unknown metabolite 'HC01104_e' created
r0590
r0595
unknown metabolite 'HC01501_c' created
r0596
r0598
unknown metabol

r1437
r1440
r1441
r1443
r1444
r1445
r1446
r1447
r1448
r1449
r1450
r1451
r1453
r1454
r1455
r1456
r1457
r1459
r1464
r1466
r1467
r1468
r1472
unknown metabolite 'HC10856_m' created
r1474
unknown metabolite 'HC10857_m' created
r1477
r1479
r1481
r1487
r1492
r1493
r1495
r1497
r1498
r1499
unknown metabolite 'HC02192_e' created
r1500
unknown metabolite 'HC02193_e' created
r1501
r1502
r1503
r1512
r1514
r1515
r1516
r1517
r1518
r1519
r1520
r1521
r1522
r1523
r1525
r1526
r1527
r1528
r1529
r1530
r1531
r1532
unknown metabolite 'HC02200_c' created
unknown metabolite 'HC02200_e' created
r1533
r1536
r1540
r1544
r1546
r1547
r1548
r1549
r1551
r1552
r1553
r1554
r1556
r1557
r1559
r1560
r1561
r1562
r1563
r1564
r1565
r1566
r1567
r1568
r1569
r1570
r1571
r1573
r1574
r1575
r1576
r1578
r1579
r1580
r1581
r1583
r1584
r1585
r1586
r1587
r1588
r1589
r1590
r1591
r1592
r1593
r1594
r1595
r1596
r1597
r1598
r1599
r1600
r1602
r1603
r1604
r1605
r1606
r1607
r1608
r1609
r1610
r1611
r1612
r1613
r1614
r1615
r1616
r1617
r1618
r161

In [5]:
##### ----- Add Boundary Reactions ----- #####
dr = []
for _, row in boundary_rxns.iterrows():
    r = Reaction(row['Reaction'])
    dr.append(r)    
model.add_reactions(dr)

boundary_rxns_dict = boundary_rxns.set_index('Reaction').to_dict()
boundary_rxns_dict

for i,r in enumerate(tqdm(model.reactions)):
    if r in dr:
        r.build_reaction_from_string(boundary_rxns_dict['Reaction Formula'][r.id])
        r.name = boundary_rxns_dict['Reaction Name'][r.id]
        r.subsystem = boundary_rxns_dict['Subsystem'][r.id]
        r.lower_bound = float(boundary_rxns_dict['Lower bound'][r.id])
        r.upper_bound = float(boundary_rxns_dict['Upper bound'][r.id]) 
model

  0%|          | 0/11668 [00:00<?, ?it/s]

Name,iCHO
Memory address,1521324d0
Number of metabolites,8004
Number of reactions,11668
Number of genes,3182
Number of groups,0
Objective expression,0
Compartments,


In [6]:
##### ----- Add information for each metabolite ----- #####
metabolites_dict = metabolites.set_index('BiGG ID').to_dict('dict')
metabolites_dict['Name']
for met in model.metabolites:
    met.name = metabolites_dict['Name'][f'{met}']
    met.formula = metabolites_dict['Formula'][f'{met}']
    met.compartment = metabolites_dict['Compartment'][f'{met}'].split(' - ')[0]    
    

### 1.3 Save and validate the model

In [22]:
##### ----- Build the S matrix ----- #####
S = cobra.util.create_stoichiometric_matrix(model, array_type='dense')
model.S = S

In [8]:
##### ----- Save the model ----- #####
write_sbml_model(model, "iCHOv3_CHO_24052023.xml")

In [9]:
##### ----- Test for errors in the recostruction ----- ######

# import tempfile
# from pprint import pprint
# from cobra.io import write_sbml_model, validate_sbml_model
# with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
#     write_sbml_model(model, filename=f_sbml.name)
#     report = validate_sbml_model(filename=f_sbml.name)
# pprint(report)

from cobra.io import read_sbml_model, validate_sbml_model
(_, errors) = validate_sbml_model("iCHOv3_CHO+Recon_v3.xml")
errors

SBML errors in validation, check error log for details.
COBRA errors in validation, check error log for details.


{'SBML_FATAL': [],
 'SBML_ERROR': ["E0 (Error): SBML component consistency (fbc, L178736); An <objective> must have one <listOfFluxObjectives>.; An <objective> object must have one and only one instance of the <listOfFluxObjectives> object. \nReference: L3V1 Fbc V3, Section 3.6\n <objective> 'obj' has no listOfFluxObjectives.\n"],
 'SBML_SCHEMA_ERROR': [],
 'SBML_WARNING': [],
 'COBRA_FATAL': [],
 'COBRA_ERROR': ['No objective coefficients in model. Unclear what should be optimized'],
 'COBRA_WARNING': [],
 'COBRA_CHECK': []}

## 2. Identification of Blocked Reactions and Dead-End Metabolites <a id='blocked&deadends'></a>
In this second part of the notebook we use two different functions from the utils module to: (1) Run a flux variability analysis and identify blocked reactions, and (2) identify dead-end metabolites. Finally we add Extracellular Exchange reactions for the dead-end metabolites that are in the extracellular compartment.

In [ ]:
##### ----- Read Model ----- #####
from cobra.io import read_sbml_model
model = read_sbml_model("iCHOv3_CHO+Recon_v3.xml")

for rxn_exchange in model.exchanges:
    rxn_exchange.bounds = (-1000, 1000)

### 2.1 Identification of Blocked Reactions
The runMinMax_Single( ) function from the utils module runs an FVA (flux variability analysis) to find the ranges of each reaction's metabolic flux at the optimum. The results are stored in the "FVA_Results.txt" file with all the reactions IDs and its respective fluxes. Reactions with no flux (**Blocked Reactions**) are stored in another file called "Blocked_Reactions.txt"

In [ ]:
##### ----- FVA ----- #####
import importlib
# from utils import runMinMax_GF
import utils
importlib.reload(utils)
model.solver = 'gurobi'


for rxn in model.reactions:
    rxn.bounds = -1000, 1000
minmax = utils.runMinMax_Single(model, end_rxn_index=None)

##### Print Reactions with the min and max fluxes #####
with open('FVA_Results.txt', 'w') as f:
    for i, j in enumerate(minmax):
        print(model.reactions[i].id, "Min: ",j[0], "Max: ", j[1], file=f)
        
##### Print Blocked Reactions #####
with open('Blocked_Reactions.txt', 'w') as f:
    for i, j in enumerate(minmax):
        if j[0] == 0 and j[1] == 0:
            print(model.reactions[i].id, "Min: ",j[0], "Max: ", j[1], file=f)
            

In [ ]:
##### ----- Print the amount  and % of blocked reactions ----- #####
blocked_reactions = []
for i, j in enumerate(minmax):
        if j[0] == 0 and j[1] == 0:
            blocked_reactions.append(model.reactions[i].id)
            
print('##### ----- Blocked Reactions ----- #####')
print(f'The model has {len(model.reactions)} total reactions')
print(f'The model has {len(blocked_reactions)} ({round(len(blocked_reactions)/len(model.reactions)*100)}%) blocked reactions')

### 2.1 Identification of Dead-Ends Metabolites
The detect_dead_ends( ) function from the utils module returns a list with all the **dead-end** metabolites in our model. A dead-end metabolite refers to a metabolite that is either only consumed but not produced, or only produced but not consumed, in a given metabolic network. The results are stored in the "Dead-ends.txt" file.

In [ ]:
##### ----- Detect Dead-Ends ----- #####
from utils import detect_dead_ends
dead_ends = detect_dead_ends(model)
model.solver = 'gurobi' #change 'gurobi' for 'glpk'

with open('Dead_ends.txt', 'w') as f:
    for i,j in enumerate(dead_ends):
        if j:
            print(model.metabolites[i], file=f)
            
rxn_c = 0
met_c = 0
for i,j in enumerate(dead_ends):
    if j:
        met_c += 1
        print("\n", model.metabolites[i],": ")
        for met_rxn in model.metabolites[i].reactions:
            rxn_c += 1
            print(met_rxn)
            print(rxn_c)

print(f'Percentage of Dead-End metabolites is {round(met_c/len(model.metabolites)*100)}%' )

### 2.3 Addition of Extracellular Exchange Reanctions
The following cell adds **EXTRACELLULAR EXCHANGE** reactions to the dead-end metabolites in the extracellular compartment from the list generated above.

In [ ]:
##### ----- Automatically add EXTRACELLULAR EXCHANGE reactions to the "BoundaryRxns" Sheet ----- #####
for i,j in enumerate(dead_ends):
    if j:
        if str(model.metabolites[i]).endswith('_e'):
            new_row_data = {'Curated': '', 'Reaction': 'EX_'+str(model.metabolites[i]), 'Reaction Name': 'Exchange of '+model.metabolites[i].name, 'Reaction Formula': str(model.metabolites[i])+' <=>', 'Subsystem': 'EXTRACELLULAR EXCHANGE',
                                    'Reversible': 1, 'Lower bound': -1000, 'Upper bound': 1000, 'Objective': 0}
            new_row_df = pd.DataFrame(new_row_data, index=[len(boundary_rxns)])
            boundary_rxns = pd.concat([boundary_rxns, new_row_df])

#Check for duplicated reactions added to the boundary_rxns dataset, IF NOT: update the google sheet file
if not boundary_rxns['Reaction'].duplicated().any() and not boundary_rxns['Reaction Formula'].duplicated().any():
    sheet.update_google_sheet(sheet_boundary, boundary_rxns)
    print("BoundaryRxns Google Sheet updated.")
else:
    print('Duplicated values found in the dataset')

In [ ]:
#from cobra.flux_analysis import gapfill
#
#recon3d = cobra.io.read_sbml_model('../Data/GPR_Curation/Recon3D.xml')
#solution = gapfill(model, recon3d, demand_reactions=False)
#
#for r in solution[0]:
#    print(r.id)

### Test CHO - Recon GEM

In [9]:
# iCHO_recon3dfrom cobra.io import read_sbml_model
# read_sbml_model(".xml")

model_EX = [i for i, rxn in enumerate(model.reactions) if 'EX_' in rxn.id]
model_SK = [i for i, rxn in enumerate(model.reactions) if 'SK_' in rxn.id]
model_DM = [i for i, rxn in enumerate(model.reactions) if 'DM_' in rxn.id]
for i in model_EX:
    model.reactions[i].bounds = -1000, 1000

for i in model_SK:
    model.reactions[i].bounds = -1000, 1000

for i in model_DM:
    model.reactions[i].bounds = 0, 1000
    

In [ ]:
model.objective = "biomass" # 
sol1 = model.optimize()
print(sol1.objective_value)

model.objective = "biomass_prod" # 
sol2 = model.optimize()
print(sol2.objective_value)

model.objective = "biomass_producing" # 
sol3 = model.optimize()
print(sol3.objective_value)


In [ ]:
##### ----- Test model KOs ----- #####
for reaction in model.reactions:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

## 3. Generation of the Mass Flow Graph <a id='MFG'></a>
Based on the publication **_Flux-dependent graphs for metabolic networks_** by _Beguerisse-Diaz et al. (2018)_ (https://www.nature.com/articles/s41540-018-0067-y). Here we use our model to build the **D Matrix** and plot the corresponding graph, then the **M Matrix** and plot the corresponding graph, and finally we generate the **PageRank** file with all the reactions in our reconstruction sorted by importance.

In [8]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from scipy.linalg import pinv
import cobra

### 3.1 Generation of the "D-Matrix"
The "D-Matrix" defines the weight of the edge between reaction nodes Ri and Rj as the probability that any metabolite chosen at random is produced by Ri (reaction i) and consumed by Rj (reaction j). Summing over all metabolites and normalizing, we obtain the edge weights of the adjacency matrix of the NFG:

In [9]:
# Load the model (Assuming SBML format)
import time
#model = cobra.io.read_sbml_model('/Users/pablodigiusto/Downloads/e_coli_core.xml')

# Extract stoichiometric matrix, m=reactions, n=metabolites
start = time.time()
S = cobra.util.array.create_stoichiometric_matrix(model)
n, m = S.shape
end = time.time()
print(f"Time taken to generate S, n and m: {end - start} seconds")

# Create S2m matrix
start = time.time()
r = np.array([reaction.reversibility for reaction in model.reactions]) # m-dimensional reversibility vector with components rj = 1 if reaction Rj is reversible and rj = 0 if it is irreversible.
Im = np.eye(m) # m × m identity matrix
diagR = r * Im # m × m matrix contains r in its main diagonal
S2m2 = np.block([[Im, np.zeros((m, m))], [np.zeros((m, m)), np.diag(r)]])
S2m1 = np.hstack((S, -S))
S2m = S2m1 @ S2m2 # unfolded version of the stoichiometric matrix of the 2m forward and reverse reactions.
end = time.time()
print(f"Time taken to generate S2m: {end - start} seconds")

# Create S2mplus and S2mminus matrices
start = time.time()
S2mplus = (np.abs(S2m) + S2m) / 2 # production stoichiometric matrix
S2mminus = (np.abs(S2m) - S2m) / 2 # consumption stoichiometric matrix
end = time.time()
print(f"Time taken to generate S2mplus and S2minus: {end - start} seconds")

# Calculate weights
start = time.time()
Wplus = pinv(np.diag(S2mplus @ np.ones((2*m, 1)).flatten()))
Wminus = pinv(np.diag(S2mminus @ np.ones((2*m, 1)).flatten()))
end = time.time()
print(f"Time taken to generate Wplus and Wminus: {end - start} seconds")

# Calculate D matrix
start = time.time()
D = 1/n * (Wplus @ S2mplus).T @ (Wminus @ S2mminus)
end = time.time()
print(f"Time taken to generate the D-Matrix: {end - start} seconds")

# Remove unused reactions
start = time.time()
IDr = np.nonzero(np.sum(D, axis=0) == 0)[0]
IDr = IDr[IDr > m]

D = np.delete(D, IDr, axis=0)
D = np.delete(D, IDr, axis=1)
end = time.time()
print(f"Time taken to remove unused reactions from the D-Matrix: {end - start} seconds")

Time taken to generate S, n and m: 0.11492776870727539 seconds
Time taken to generate S2m: 125.76139998435974 seconds
Time taken to generate S2mplus and S2minus: 2.972075939178467 seconds
Time taken to generate Wplus and Wminus: 406.39779591560364 seconds
Time taken to generate the D-Matrix: 207.65615105628967 seconds
Time taken to remove unused reactions from the D-Matrix: 2.4509801864624023 seconds


In [13]:
i = 0
for r in model.reactions:
    if r.reversibility:
        i+=2
    elif not r.reversibility:
        i+=1
        
print(i)

16056


In [17]:
S2m.shape

(8004, 23336)

### 3.2 Plotting the "D-Matrix", Normalized Flow Graph (NFG)

In [24]:
# Convert D matrix into a graph
G = nx.from_numpy_array(D, create_using=nx.DiGraph)
weights = [G[u][v]['weight'] for u,v in G.edges()]
normalized_weights = [(w - min(weights)) / (max(weights) - min(weights)) * (15 - 0.0001) + 0.0001 for w in weights]



# Create a dictionary mapping node indices to reaction names for labeling
reaction_names = [r.id for r in model.reactions]
node_labels = {i: reaction_name for i, reaction_name in enumerate(reaction_names)}

# Calculate the degrees of each node
degrees = [val for (node, val) in G.degree()]

# Normalize the degrees for color mapping
normalized_degrees = [(d - min(degrees)) / (max(degrees) - min(degrees)) for d in degrees]

# Use a colormap to map normalized degrees to colors
cmap = plt.get_cmap('OrRd')  # Choose a colormap here
node_colors = [cmap(deg) for deg in normalized_degrees]


# Plot the graph
fig, ax = plt.subplots(figsize=(20, 20))
pos = nx.spring_layout(G)
nx.draw(G, pos, width=normalized_weights, node_color=node_colors, edge_color='lightgray', node_size=700, ax=ax)

# Labels
label_pos = {node: (x + 0.01, y + 0.03) for node, (x, y) in pos.items()}
nx.draw_networkx_labels(G, label_pos, labels=node_labels, font_size=17, ax=ax)

plt.title('Network Flux Graph (NFG)')
plt.show()

KeyboardInterrupt: 

### 3.3 Generation of the "FluxOpenValue" matrix

In [63]:
# Load the model (Assuming SBML format)
# Perform FBA for Biomass on Non-Producing and Producing Cell Lines
objectives = ['biomass', 'biomass_producing']

fluxes_list = []
for objective in objectives:
    model.objective = objective
    solution = model.optimize()
    fluxes = np.array(solution.fluxes)
    fluxes_list.append(fluxes)
    
# Stack arrays horizontally
FluxOpenValue = np.column_stack(fluxes_list)

FluxOpenValue

array([[ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [-5.90558363e-02, -1.96858994e-02],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.

### 3.4 Generation of the "Mass Flow Graph (MFG) Matrix"
The MFG provides a versatile framework to create environment-specific metabolic graphs from FBA solutions

In [64]:
# Calculation of the MFG
PageRank = []
for i in range(FluxOpenValue.shape[1]):
    start = time.time()
    v1 = FluxOpenValue[:, i].T

    # unfolding the flux vector
    # creation of vplus and vminus
    vplus = (np.abs(v1) + v1) / 2
    vminus = (np.abs(v1) - v1) / 2

    # creation of v2m
    v2m = np.concatenate((vplus, vminus))

    # creation of J_v
    J_v = S2mplus @ v2m.reshape(-1)

    # calculation of the MFG
    M = (S2mplus * v2m).T @ pinv(np.diag(J_v)) @ (S2mminus * v2m)
    

    # Calculate and store PageRank
    G = nx.from_numpy_array(M, create_using=nx.DiGraph)
    pr = nx.pagerank(G)
    PageRank.append(pr)
    end = time.time()
    print(f"Time taken to go through iteration {i}: {end - start} seconds")
    

# Post-processing of PageRank
df = pd.DataFrame(PageRank)
PageRank = df.values
PageRank = np.array(PageRank).T
PageRankRxns = PageRank[:m, :]
PageRankRxns_back = PageRank[m:, :]

for i in range(m):
    for j in range(PageRankRxns.shape[1]):
        if PageRankRxns_back[i, j] > PageRankRxns[i, j]:
            PageRankRxns[i, j] = PageRankRxns_back[i, j]

Time taken to go through iteration 0: 373.80980610847473 seconds
Time taken to go through iteration 1: 387.8797118663788 seconds


In [87]:
row_sums = PageRankRxns.sum(axis=1)
sorted_indices = np.argsort(row_sums)
for i in sorted_indices[::-1]:
    print(model.reactions[i])

NACPRT: atp_c + h2o_c + nac_c + prpp_c <=> adp_c + nicrnt_c + pi_c + ppi_c
DMNONCOAtcx: atp_c + dmnoncoa_c + h2o_c <=> adp_c + dmnoncoa_x + h_c + pi_c
OCCOAtcx: atp_c + h2o_c + occoa_c <=> adp_c + h_c + occoa_x + pi_c
RE3238C: CE5151_c + h2o_c <=> CE2510_c + coa_c + h_c
BiGGRxn67: h2o_c + udpgal_c <=> gal1p_c + 2.0 h_c + ump_c
r1154: 2obut_m + coa_m + nad_m <=> co2_m + nadh_m + ppcoa_m
r0034: 2.0 atp_m + co2_m + h2o_m + nh4_m --> 2.0 adp_m + cbp_m + 3.0 h_m + pi_m
HMR_0293: CE2510_c + atp_c + coa_c <=> CE5151_c + amp_c + ppi_c
r0643: 2mop_m + h2o_m + nad_m <=> HC00900_m + 2.0 h_m + nadh_m
HCO3E: co2_c + h2o_c <=> h_c + hco3_c
MTHFR3: 2.0 h_c + mlthf_c + nadph_c --> 5mthf_c + nadp_c
UGLT: gal1p_c + udpg_c <=> g1p_c + udpgal_c
r0391: nicrnt_c + ppi_c <=> h_c + nac_c + prpp_c
r0295: co2_m + mlthf_m + nadh_m + nh4_m --> gly_m + nad_m + thf_m
r0280: dadp_c + h_c + pep_c --> datp_c + pyr_c
GapFill-R08726: 2.0 h_c + 2.0 nadph_c + 2.0 o2_c + xol27oh_c <=> CE1292_c + 3.0 h2o_c + 2.0 nadp_c
NDPK

In [93]:
PageRankRxns[1956:,:]

array([[3.62684815e-04, 3.63773380e-04],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [1.21536508e-04, 1.21494321e-04],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [2.11797362e-04, 2.11772644e-04],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.17253865e-05, 4.17254009e-05],
       [4.172538

In [86]:
sorted_indices[::-1]

array([ 6040,  1956,  6242,  7753,  1274,  9660,  9221,  3620,  9417,
        3422,  5977,  9026,  9317,  9283,  9279,  3344,  6123,  9072,
         842,  1974,  1917,  6856,  6144,  9551,  2844,  6793,  5351,
        5352,  5640,  9387,  1540,  7609,  9524,  5034,  4185,  1488,
        6113,  5864,  6285,   361,  7248,  3031,  6629,  9047, 10137,
        5457,  6077,  3206,  9295,  7804,  9071,  3711,  6692,  1698,
        4969,  4017,  6854,  1271,   768,  8114,  1711,  2663,  1366,
        9107, 11129,  9228,  5612,  6727,  6290,  1266,  9204,  1746,
        3302,  9202,  6289,   302,  6898,  8708,  9559,  3123,  9297,
        8460,  4018,  6693,  3127,  8530,  6852,  3354,  9918,  1442,
        8853,  9024,  1107,  7167,  9545,  1667,  1724,  9118,  3310,
         830,  9746,  1535, 10545,  9106, 11042,  1233,  6686,  1717,
        3276,  1970,  9088,  1675,  4325,  9396, 10355,  3277, 10188,
        8331,  6548,  1482,  3211,  1873,  9250,  9774, 11276,  9232,
        3237,  8143,

In [101]:
PageRankRxns_sorted_descending = PageRankRxns[sorted_indices[::-1]]
PageRankRxns_sorted_descending[400:,]

array([[4.80700719e-05, 4.80700390e-05],
       [4.70383806e-05, 4.70165183e-05],
       [4.74579856e-05, 4.65327283e-05],
       [4.68785840e-05, 4.68611649e-05],
       [4.67939464e-05, 4.68107774e-05],
       [4.55903862e-05, 4.55861259e-05],
       [4.55546473e-05, 4.53329687e-05],
       [4.56082449e-05, 4.49565601e-05],
       [4.52819932e-05, 4.52820070e-05],
       [4.49582990e-05, 4.49502746e-05],
       [4.46186098e-05, 4.46223034e-05],
       [4.45044276e-05, 4.45098185e-05],
       [4.45042025e-05, 4.43511909e-05],
       [4.34676085e-05, 4.34262120e-05],
       [4.33320250e-05, 4.32443619e-05],
       [4.21541119e-05, 4.21537628e-05],
       [4.22785251e-05, 4.19246231e-05],
       [4.20946968e-05, 4.20947113e-05],
       [4.20681785e-05, 4.18472520e-05],
       [4.19262381e-05, 4.17971597e-05],
       [4.18756222e-05, 4.17790854e-05],
       [4.18624807e-05, 4.17735220e-05],
       [4.18280149e-05, 4.17619684e-05],
       [4.17962297e-05, 4.17501900e-05],
       [4.179412

In [65]:
np.savetxt('PageRankRxns.txt', PageRankRxns, fmt='%.10f')

### 3.5 Plotting the "MFG Matrix"

In [ ]:
# Plot the graph
G = nx.from_numpy_array(M, create_using=nx.DiGraph)
weights = [G[u][v]['weight'] for u,v in G.edges()]
normalized_weights = [(w - min(weights)) / (max(weights) - min(weights)) * (15 - 0.001) + 0.001 for w in weights]

#Layout
pos = nx.random_layout(G)

# Calculate the degrees of each node
degrees = [val for (node, val) in G.degree()]

# Normalize the degrees for color mapping
normalized_degrees = [(d - min(degrees)) / (max(degrees) - min(degrees)) for d in degrees]

# Use a colormap to map normalized degrees to colors
cmap = plt.get_cmap('OrRd')  # Choose a colormap here
node_colors = [cmap(deg) for deg in normalized_degrees]


labels = np.concatenate(([reaction.id for reaction in model.reactions],
                         [reaction.id + '_r' for reaction in model.reactions]))
node_labels = {i: reaction_name for i, reaction_name in enumerate(labels)}


fig, ax = plt.subplots(figsize=(20, 20))  # Create a new figure and axes
nx.draw(G, pos, node_color=node_colors, node_size=500, edge_color='lightgray',
        edge_cmap=plt.cm.Blues, width=normalized_weights, ax=ax)
label_pos = {node: (x + 0.01, y + 0.03) for node, (x, y) in pos.items()}
nx.draw_networkx_labels(G, label_pos, labels=node_labels, font_size=12, ax=ax)

plt.title('MFG')
plt.show()

### Identification of duplicates throught Chemical Formulas

In [ ]:
# Chemical Formula
from cobra.io import read_sbml_model
model = read_sbml_model("iCHOv3_CHO_23052023.xml")

import pandas as pd
data = []

for m in model.metabolites:
    m.name = m.formula
for r in model.reactions:
    if r not in model.exchanges:
        print(r)
        reaction_id = r.id
        reaction_name = r.name
        reaction_formula = r.build_reaction_string(use_metabolite_names=True)
        data.append([reaction_id, reaction_name, reaction_formula])
df = pd.DataFrame(data, columns=['ID', 'Name', 'Reaction'])
df

In [ ]:
for m in model.metabolites:
    m.name = m.formula
for r in model.reactions:
    if r not in model.exchanges:
        reaction_id = r.id
        reaction_name = r.name
        reaction_formula = r.build_reaction_string(use_metabolite_names=True)
        data.append([reaction_id, reaction_name, reaction_formula])
df = pd.DataFrame(data, columns=['ID', 'Name', 'Reaction'])
df

In [16]:
subset_duplicates = df[df.duplicated(subset='Reaction', keep=False)].reset_index(drop=True)
subset_duplicates = subset_duplicates.sort_values(by=['Reaction'])
subset_duplicates.to_excel('Test.xlsx')

for index, n in enumerate(subset_duplicates['Reaction']):
    next_value = subset_duplicates['Reaction'][index + 1]
    if n == next_value:
        print(subset_duplicates['Name'][index])

        

### Duplicates identified

In [32]:
print(model.metabolites.get_by_id('uppg1_c').name, model.metabolites.get_by_id('HC01609_c').name)
for r in model.metabolites.get_by_id('uppg1_c').reactions:
    print(r)
print('------')
for r in model.metabolites.get_by_id('HC01609_c').reactions:
    print(r)
# Based on reations HMR_4772 and GapFill-R04972, HC01609_c and uppg1_c  are the same 

C40H36N4O16 C40H36N4O16
GapFill-R03166: hmbil_c <=> h2o_c + uppg1_c
GapFill-R04972: uppg1_c <=> 4.0 co2_c + cpppg1_c
------
C40H36N4O16
HMR_4772: HC01609_c + 4.0 h_c --> 4.0 co2_c + cpppg1_c


In [31]:
print(model.metabolites.get_by_id('HC02187_c').name, model.metabolites.get_by_id('triodthy_c').name)
for r in model.metabolites.get_by_id('HC02187_c').reactions:
    print(r)
print('------')
for r in model.metabolites.get_by_id('triodthy_c').reactions:
    print(r)
# Based on reations HMR_6834 and HMR_6826, triodthy_c and HC02187_c  are the same 

C15H12I3NO4 C15H12I3NO4
HMR_6834: CE2866_c + i_c + nadp_c <=> HC02187_c + nadph_c
r2196: HC02187_e + gthrd_c + hco3_c <=> HC02187_c + gthrd_e + hco3_e
HMR_6839: h2o_c + triodthysuf_c --> HC02187_c + h_c + so4_c
r2213: HC02187_e + HC02200_c + hco3_c <=> HC02187_c + HC02200_e + hco3_e
HMR_6838: HC02187_c + paps_c --> h_c + pap_c + triodthysuf_c
r2352: HC02187_e <=> HC02187_c
r2214: HC02187_e + HC02201_c + hco3_c <=> HC02187_c + HC02201_e + hco3_e
r2212: HC02187_e + HC02199_c + hco3_c <=> HC02187_c + HC02199_e + hco3_e
------
C15H12I3NO4
r2210: HC02200_c + hco3_c + triodthy_e <=> HC02200_e + hco3_e + triodthy_c
r2209: HC02199_c + hco3_c + triodthy_e <=> HC02199_e + hco3_e + triodthy_c
TIDSSULF: h2o_c + triodthysuf_c --> h_c + so4_c + triodthy_c
r2211: HC02201_c + hco3_c + triodthy_e <=> HC02201_e + hco3_e + triodthy_c
RE0922C: triodthy_c + udpglcur_c --> CE2879_c + h_c + udp_c
TRIODTHYSULT: paps_c + triodthy_c --> h_c + pap_c + triodthysuf_c
HMR_6840: CE2872_c + i_c + nadp_c <=> nadph_c +